# Init

In [112]:
import pandas as pd
from bert_score import BERTScorer
import csv

import eval_util

In [10]:
scorer = BERTScorer(lang='en', rescale_with_baseline=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# SR

In [110]:
sr_csv = 'augment/data/sr_augmented.csv'
sr_df = pd.read_csv(sr_csv)
print(sr_df.shape)
sr_df.head()

(5832, 3)


,Tweet,Target,Stance
0,He who exalts himself shall be humbled; and he...,Atheism,0
1,He WHO exalts himself shall be humbled ; and h...,Atheism,0
2,He WHO exalts himself shall be humbled ; and a...,Atheism,0
3,RT @prayerbullets: I remove Nehushtan -previou...,Atheism,0
4,RT @ prayerbullets : I remove Nehushtan -previ...,Atheism,0


In [98]:
i = 600
assert i % 3 == 0

print(sr_df.iloc[i]['Tweet'])
print(sr_df.iloc[i+1]['Tweet'])
print(sr_df.iloc[i+2]['Tweet'])

print(eval_util.get_bertscore(scorer,
                              sr_df.iloc[i]['Tweet'],
                              [sr_df.iloc[i+1]['Tweet'],
                               sr_df.iloc[i+2]['Tweet']]))

I pray for the best for everybody ! ! ! # SemST
@Brasilmagic @neiltyson there's no god 2b tired of, just tired o ppl believing in such nonsense. Time 2 focus on reality, HUMANITY #SemST
@ Brasilmagic @ neiltyson there 's No god 2b tired of , just tired o ppl believing in such nonsense . Time two focus on reality , HUMANITY # SemST
tensor([-0.0363, -0.0207])


In [120]:
# detecting errors in sr csv

i = 1185

while i < sr_df.shape[0]:
    score = eval_util.get_bertscore(scorer,
                                    sr_df.iloc[i]['Tweet'],
                                    [sr_df.iloc[i+1]['Tweet'],
                                     sr_df.iloc[i+2]['Tweet']])
    if score[0] < 0.3 or score[1] < 0.3:
        print(i)
        print(sr_df.iloc[i]['Tweet'])
        print(sr_df.iloc[i+1]['Tweet'])
        print(sr_df.iloc[i+2]['Tweet'])
        print(f'{score=}')
        break
    i += 3

1200
Husband: "They should know that she's always born with it. It's never Maybelline." #beauty #SemST
Husband : `` They should get laid that she 's always Born with it . information technology 's ne'er Maybelline . '' # mantrap # SemST
husband : `` They should bed that she 's incessantly born with it . It 's never Maybelline . '' # beauty # SemST
score=tensor([0.2535, 0.4795])


In [ ]:
# calculate bertscores for sr csv. each tweet has 2 sr augmentations, so
# tweets are grouped into 3s. scores generated by pairs, so scores in written
# file are grouped into 2s
# score for first 1000 original tweets

sr_scores = []

for i in range(0, sr_df.shape[0], 3):
